# Cycle
This cycle uses mixture experimentalist, BMS theorist, and equation sampler as a source for the ground truth. 

In [57]:
import copy
from dataclasses import dataclass, field
from typing import List

import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, max_error
import numpy as np
import math
import pandas as pd
from autora.variable import VariableCollection, Variable
from autora.state.standard import StandardState
from autora.state import on_state
from autora.state.wrapper import state_fn_from_estimator
from autora.theorist.bms import BMSRegressor
from equation_tree import sample
from equation_tree.sample import sample_fast
from equation_tree.tree import instantiate_constants
from equation_tree.prior import DEFAULT_PRIOR_FUNCTIONS, DEFAULT_PRIOR_OPERATORS, \
    structure_prior_from_max_depth, structure_prior_from_depth, priors_from_space
import pprint
from autora.experiment_runner.synthetic.abstract.equation import equation_experiment
#from autora.experimentalist.mixture import sample as mixture_sample
#from autora.experimentalist.grid_ import grid_pool
from autora.experimentalist.random_ import random_sample, random_pool
from autora.state import Delta
from autora.experimentalist.falsification import falsification_score_sample
from autora.experimentalist.model_disagreement import model_disagreement_score_sample
from autora.experimentalist.novelty import novelty_score_sample

In [62]:
# SAMPLING
N_CONDITIONS = 50000
NUM_SAMPLES = 100
POOL_RANGE = 5
TEMPERATURE_0 = 10

# EVALUATION
NUM_EVAL_SAMPLES = 10000

# EQUATION
MAX_NUM_VARIABLES = 10
NUM_POOL_SAMPLES = 10_000
CONSTANT_SIZE = 5

# SIMULATION
EPOCHS = 100
CYCLES = 100

In [28]:
# SAMPLING CONSTANTS FOR THE SIMULATIONS

def temp_sample(n):
    sampled_values = []
    for _ in range(n):
        # Choose the range to sample from
        range_choice = np.random.choice(['0-0.25', '0.25-10'], p=[0.5, 0.5])
        
        if range_choice == '0-0.25':
            sampled_value = np.random.uniform(0, 0.25)
        else:
            sampled_value = np.random.uniform(0.25, 10)
        
        sampled_values.append(sampled_value)
    
    return sampled_values

TEMPERATURE = temp_sample(1)[0]

ANNEALING = bool(np.random.choice([False, True], 1))

falsification_weight = np.random.rand(1)[0]
novelty_weight = np.random.rand(1)[0]
disagreement_weight = np.random.rand(1)[0]
WEIGHTS = {'falsification': falsification_weight, 'novelty':novelty_weight, 'disagreement': disagreement_weight}

TREE_DEPTH = np.random.choice([3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])

N_DEAD_VARS = int(np.floor(np.random.gamma(1, 2, 1)))
DEAD_VARIABLES_ = ["d_1", "d_2", "d_3", "d_4", "d_5", "d_6", "d_7", "d_8", "d_9", "d_10", "d_11", "d_12", "d_13", "d_14", "d_15"][:N_DEAD_VARS]
DEAD_VARIABLES = []
for d in DEAD_VARIABLES_:
    DEAD_VARIABLES.append(Variable(d,value_range=(-POOL_RANGE, POOL_RANGE)))

ADDED_NOISE = np.random.rand(1)[0] * 2

In [29]:
def cos_annealed_t(T_0, cycles, i):
    """
    Args:
        T_0: Start temperature
        cycles: number of cycles
        i: current cycle
    """

    return T_0 * math.cos(i * math.pi / (2 * cycles))

## Ground truth
Sampling the ground truth for this simulation.

In [32]:
feature_prior = {'constants': .5, 'variables': .5}
operator_pior = priors_from_space(["+", "-", "*", "/", "^"])
if TREE_DEPTH <= 8:
    structure_prior = structure_prior_from_depth(TREE_DEPTH) ### CHANGE THIS NUMBER TO LOOK AT THE EQUATIONS WITH DIFFERENT TREE DEPTH
    prior = {'functions': DEFAULT_PRIOR_FUNCTIONS, 'operators': operator_pior, 'structures': structure_prior, 'features': feature_prior}
    sampled_equations = sample(n=45, prior=prior, max_num_variables=2)
else:
    prior = {'functions': DEFAULT_PRIOR_FUNCTIONS, 'operators': operator_pior, 'features': feature_prior}
    print("HERE")
    sampled_equations = sample_fast(n=45, prior=prior, tree_depth=TREE_DEPTH, max_num_variables=2)

fig = plt.figure(figsize=(12, 32))
fig.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.5, wspace=0.4)


for n in range(len(sampled_equations)):
    equation = sampled_equations[n]
    equation = instantiate_constants(equation, lambda: np.random.rand()*CONSTANT_SIZE)
    #print(equation.sympy_expr)
    features = {}
    for v in range(equation.n_variables_unique):
        array = np.linspace(-POOL_RANGE, POOL_RANGE, 1000)
        np.random.shuffle(array)
        features[equation.variables_unique[v]] = array
    
    variable_names = list(features.keys())        

    
    result = equation.evaluate(features)
    #plt.plot(features['x_1'],result.iloc[:], 'o')
    if len(variable_names) == 1:
        ax = fig.add_subplot(15,3,n+1)
        ax.set_title(equation.sympy_expr)
        ax.plot(features[variable_names[0]], result, '.', markersize=1)
    else: 
        ax = fig.add_subplot(15,3,n+1, projection='3d')
        ax.set_title(equation.sympy_expr)
        ax.plot(features[variable_names[0]], features[variable_names[1]], result, '.', markersize=1)

    
    #ax.plot3D(features['x_1'], features['x_2'], result.iloc[:], 'o', s=1)


HERE


Processing:  31%|███       | 14/45 [07:58<17:39, 34.19s/iteration]


KeyboardInterrupt: 

In [33]:
if TREE_DEPTH <= 8:
    structure_prior = structure_prior_from_depth(TREE_DEPTH) ### CHANGE THIS NUMBER TO LOOK AT THE EQUATIONS WITH DIFFERENT TREE DEPTH
    prior = {'functions': DEFAULT_PRIOR_FUNCTIONS, 'operators': operator_pior, 'structures': structure_prior, 'features': feature_prior}
    equation_raw = sample(n=1, prior=prior, max_num_variables=2)
else:
    prior = {'functions': DEFAULT_PRIOR_FUNCTIONS, 'operators': operator_pior, 'features': feature_prior}
    equation_raw = sample_fast(n=1, prior=prior, tree_depth=TREE_DEPTH, max_num_variables=2)
equation_raw[0].sympy_expr

Processing: 100%|██████████| 1/1 [00:37<00:00, 37.43s/iteration]


(c_1 - asin(sin(x_1)) + (c_2*sin(x_1) - c_3)/c_4)**c_5

In [34]:
equation = instantiate_constants(equation_raw[0], lambda: np.random.rand()*CONSTANT_SIZE)
equation.sympy_expr
#equation_raw[0].sympy_expr

(0.38015063232559247*sin(x_1) - asin(sin(x_1)) + 0.5243878999193152)**3.0552585076906693

In [35]:
features = {}
for v in range(equation.n_variables_unique):
    array = np.linspace(-POOL_RANGE, POOL_RANGE, 1000)
    np.random.shuffle(array)
    features[equation.variables_unique[v]] = array

Defining the metadata based on the sampled ground truth.

In [36]:
independent_variables = []
for v in range(equation.n_variables_unique):
    independent_variables.append(Variable(equation.variables_unique[v],value_range=(-POOL_RANGE, POOL_RANGE)))


variables_for_the_experiment=VariableCollection(
        independent_variables=independent_variables,
        dependent_variables=[Variable("y")]
    )

independent_variables_ = independent_variables.copy()
#variables with dead variables
if len(DEAD_VARIABLES)>0:
    for i in range(len(DEAD_VARIABLES)):
        independent_variables_.append(DEAD_VARIABLES[i])
        
variables=VariableCollection(
        independent_variables=independent_variables_,
        dependent_variables=[Variable("y")]
    )

Defining experiment runner from the equation and the variable collection

In [37]:
experiment = equation_experiment(equation.sympy_expr, variables_for_the_experiment.independent_variables, variables_for_the_experiment.dependent_variables[0], rename_output_columns=False)

### Defining the state
We can define an initial state for our discovery problem based on the variable specification above. Wrapping experiment runner into the state.

In [48]:
@dataclass(frozen=True)
class ExtendedState(StandardState):
    models_bms: List[BaseEstimator] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    )
    models_linear: List[BaseEstimator] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    )
    models_polynom: List[BaseEstimator] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    )
    rejections: List[int] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    )
    mad: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    )
    cycle_step: int = field(
        default_factory=int,
        metadata={"delta": "replace"},
    )
    mad_bms_oos: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    mad_linear_oos: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    mad_poly_oos: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    msd_bms_oos: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    msd_linear_oos: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    msd_poly_oos: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    mxe_bms_oos: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    mxe_linear_oos: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    mxe_poly_oos: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    mad_bms_is: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    mad_linear_is: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    mad_poly_is: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    msd_bms_is: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    msd_linear_is: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    msd_poly_is: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    mxe_bms_is: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    mxe_linear_is: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 
    mxe_poly_is: List[float] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    ) 

state = ExtendedState(
    variables=variables
)
runner_on_state = on_state(experiment.experiment_runner, output=["experiment_data"])

### Pooler

In [49]:
@on_state()
def experimentalist_pooler(variables, equation, dead_variables = None):
    column_names = [v.name for v in variables.independent_variables]
    if dead_variables:
        column_names += dead_variables
    
    conditions_ = pd.DataFrame(columns=column_names)

    i = 0
    n = 0
    while i < 1_000_000 and len(conditions_.index) < NUM_POOL_SAMPLES:
        _sample = random_pool(variables, NUM_POOL_SAMPLES)
        if dead_variables:
            _dead_sample = random_pool(dead_variables, NUM_POOL_SAMPLES)
            _sample = pd.concat([_sample, _dead_sample], axis=1)
       
        n += NUM_POOL_SAMPLES
        evaluation = equation.evaluate(_sample)
        bad_indices = np.where(np.isnan(evaluation) | np.isinf(evaluation))[0]
        _sample = _sample.drop(bad_indices)
        if np.isnan(evaluation).any() or np.isinf(evaluation).any():
            i+=len(bad_indices)
        conditions_ = pd.concat([conditions_,_sample], ignore_index=True)
    if i >= 1_000_000:
        return None
    conditions_ = conditions_.head(NUM_POOL_SAMPLES)
    
    
    return Delta(conditions=conditions_, rejections=[i/n])

## Mixture experimentalist
Defining the mixture experimentalist and wrapping it into the state

In [50]:
"""
Mixture Experimentalist Sampler
"""

import numpy as np
from typing import Optional, Union

import pandas as pd


def adjust_distribution(p_, temperature):
    # temperature cannot be 0
    assert temperature != 0, 'Temperature cannot be 0'
    p = np.array(p_)
    # If the temperature is very low (close to 0), then the sampling will become almost deterministic, picking the event with the highest probability.
    # If the temperature is very high, then the sampling will be closer to uniform, with all events having roughly equal probability.

    p = p / np.sum(np.abs(p))  # Normalizing the initial distribution

    p = np.exp(p / temperature)
    final_p = p / np.sum(p)  # Normalizing the final distribution
    return final_p


def sample(conditions: Union[pd.DataFrame, np.ndarray], temperature: float,
                   samplers: list, params: dict,
                   num_samples: Optional[int] = None) -> pd.DataFrame:
    """

    Args:
        conditions: pool of experimental conditions to evaluate: pd.Dataframe
        temperature: how random is selection of conditions (cannot be 0; (0:1) - the choices are more deterministic than the choices made wrt
        samplers: tuple containing sampler functions, their names, and weights
        for sampler functions that return both positive and negative scores, user can provide a list with two weights: the first one will be applied to positive scores, the second one -- to the negative
        params: nested dictionary. keys correspond to the sampler function names (same as provided in samplers),
        values correspond to the dictionaries of function arguments (argument name: its value)
        num_samples: number of experimental conditions to select

    Returns:
        Sampled pool of experimental conditions with the scores attached to them
    """

    condition_pool = pd.DataFrame(conditions)

    rankings = pd.DataFrame()
    mixture_scores = np.zeros(len(condition_pool))
    ## getting rankings and weighted scores from each function
    for (function, name, weight) in samplers:

        sampler_params = params[name]
        pd_ranking = function(conditions=condition_pool, **sampler_params)

        # except:
        #     pd_ranking = function(conditions=condition_pool)
        # sorting by index
        pd_ranking = pd_ranking.sort_index()
        # if only one weight is provided, use it for both negative and positive dimensions
        if isinstance(weight, float) or isinstance(weight, int):
            pd_ranking["score"] = pd_ranking["score"] * weight
        else:
            if len(pd_ranking["score"] < 0) > 0 and len(pd_ranking["score"] > 0) > 0:  # there are both positive and negative values
                pd_ranking.loc[pd_ranking["score"] > 0]["score"] = pd_ranking.loc[pd_ranking["score"] > 0]["score"] * weight[0]  # positive dimension gets the first weight
                pd_ranking.loc[pd_ranking["score"] < 0]["score"] = pd_ranking.loc[pd_ranking["score"] < 0]["score"] * weight[1]  # negative dimension gets the second weight
            else:
                pd_ranking["score"] = pd_ranking["score"] * weight[0]

        pd_ranking.rename(columns={"score": f"{name}_score"}, inplace=True)
        # sum_scores are arranged based on the original conditions_ indices
        mixture_scores = mixture_scores + pd_ranking[f"{name}_score"]

        rankings = pd.merge(rankings, pd_ranking, left_index=True, right_index=True, how="outer")

    # adjust mixture scores wrt temperature
    weighted_mixture_scores_adjusted = adjust_distribution(mixture_scores, temperature)

    if num_samples is None:
        num_samples = condition_pool.shape[0]

    condition_indices = np.random.choice(np.arange(len(condition_pool)), num_samples,
                                         p=weighted_mixture_scores_adjusted, replace=False)
    conditions_ = condition_pool.iloc[condition_indices]
    conditions_["score"] = mixture_scores

    return conditions_


mixture_sample_test = sample

In [51]:
def get_best_model(models, X, y):
    mads = []
    for m in models:
        prediction = m.predict(X)
        mad = mean_absolute_error(y, prediction)
        mads.append(mad)
    min_value = min(mads)
    min_index = mads.index(min_value)
    mads[min_index] = math.inf
    min_value_second = min(mads)
    min_index_second = mads.index(min_value_second)
    return models[min_index], models[min_index_second], min_value




In [52]:
@on_state()
def experimentalist_sample(conditions,
                           models,
                           models_bms,
                           models_linear,
                           models_polynom,
                           experiment_data,
                           variables,
                           temperature,
                           weights,
                           num_samples):
    if models is None or experiment_data is None:
        print('First cycle: Using random sampler')
        conditions_ = random_sample(conditions, num_samples)
        mad = None
    else:
        experiment_conditions = experiment_data[[v.name for v in variables.independent_variables]]
        experiment_observations = experiment_data[[v.name for v in variables.dependent_variables]]
        params_ = {} #copy.deepcopy(params)
        params_["falsification"] = {"reference_conditions": experiment_conditions, "reference_observations": experiment_observations, "model": models[-1], "num_samples": NUM_POOL_SAMPLES}
        params_["novelty"] = {"reference_conditions": experiment_conditions, "num_samples": NUM_POOL_SAMPLES}
        
        second_bms_model = models_bms[-1].get_models()[1]
        models_to_consider = [models_bms[-1], second_bms_model, models_linear[-1], models_polynom[-1]]
        best_model, second_best_model, mad = get_best_model(models_to_consider, experiment_conditions, experiment_observations)
        
        params_["disagreement"] = {"models": [best_model, second_best_model], "num_samples": NUM_POOL_SAMPLES}

        samplers = [
            [novelty_score_sample, "novelty", weights["novelty"]],
            [falsification_score_sample, "falsification", weights["falsification"]],
            [model_disagreement_score_sample, "disagreement", weights["disagreement"]]
        ]

        conditions_ = mixture_sample_test(conditions, temperature, samplers, params_, num_samples)
        conditions_ = conditions_.drop("score", axis = 1)
    #d = Delta(conditions=conditions)
    d = Delta(conditions = conditions_, mads=[mad])
    return d

## BMS theorist
Defining the BMS theorist and wrapping it into the state

In [53]:
@on_state()
def bms_theorist(experiment_data: pd.DataFrame, variables: VariableCollection, **kwargs):
    ivs = [v.name for v in variables.independent_variables]
    dvs = [v.name for v in variables.dependent_variables]
    X, y = experiment_data[ivs], experiment_data[dvs]
    new_model = BMSRegressor(epochs=EPOCHS).set_params(**kwargs).fit(X, y)
    return Delta(models_bms=[new_model])

@on_state()
def linear_theorist(experiment_data: pd.DataFrame, variables: VariableCollection, **kwargs):
    ivs = [v.name for v in variables.independent_variables]
    dvs = [v.name for v in variables.dependent_variables]
    X, y = experiment_data[ivs], experiment_data[dvs]
    new_model = LinearRegression().set_params(**kwargs).fit(X, y)
    return Delta(models_linear=[new_model])


def PolynomialRegression(degree=3, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))


@on_state()
def polynomial_theorist(experiment_data: pd.DataFrame, variables: VariableCollection, **kwargs):
    ivs = [v.name for v in variables.independent_variables]
    dvs = [v.name for v in variables.dependent_variables]
    X, y = experiment_data[ivs], experiment_data[dvs]
    new_model = PolynomialRegression()
    new_model.fit(X, y)
    return Delta(models_polynom=[new_model])


Model evaluation

In [67]:
@on_state()
def best_model(models_bms, models_linear, models_polynom, experiment_data, variables, cycle_step):
    ivs = [v.name for v in variables.independent_variables]
    dvs = [v.name for v in variables.dependent_variables]
    X, y = experiment_data[ivs], experiment_data[dvs]
    prediction_bms = models_bms[-1].predict(X)
    prediction_linear = models_linear[-1].predict(X)
    prediction_polynomial = models_polynom[-1].predict(X)
    mad_bms = mean_absolute_error(y, prediction_bms)
    mad_linear = mean_absolute_error(y, prediction_linear)
    mad_poly = mean_absolute_error(y, prediction_polynomial)
    msd_bms = mean_squared_error(y, prediction_bms)
    msd_linear = mean_squared_error(y, prediction_linear)
    msd_poly = mean_squared_error(y, prediction_polynomial)
    mxe_bms = max_error(y, prediction_bms)
    mxe_linear = max_error(y, prediction_linear)
    mxe_poly = max_error(y, prediction_polynomial)
    if msd_bms <= msd_linear and msd_bms <= msd_poly:
        new_model = models_bms[-1]
    elif msd_linear <= msd_bms and msd_linear <= msd_poly:
        new_model = models_linear[-1]
    elif msd_poly <= msd_linear and msd_poly <= msd_bms:
        new_model = models_polynom[-1]
    cycle_step_ = cycle_step + 1
    
    return Delta(model=new_model, cycle_step = cycle_step_, mad_bms_is = [mad_bms], mad_linear_is = [mad_linear], mad_poly_is = [mad_poly], msd_bms_is = [msd_bms], msd_linear_is = [msd_linear], msd_poly_is = [msd_poly], mxe_bms_is = [mxe_bms], mxe_linear_is = [mxe_linear], mxe_poly_is = [mxe_poly])

@on_state()
def evaluation_oos(models_bms, models_linear, models_polynom, variables, dead_variables=None):
    ivs = [v.name for v in variables.independent_variables]
    if dead_variables:
        ivs += dead_variables
    dvs = [v.name for v in variables.dependent_variables]

    conditions_ = pd.DataFrame(columns=ivs)

    i = 0
    n = 0
    while i < 1_000_000 and len(conditions_.index) < NUM_EVAL_SAMPLES:
        _sample = random_pool(variables, NUM_EVAL_SAMPLES)
        if dead_variables:
            _dead_sample = random_pool(dead_variables, NUM_EVAL_SAMPLES)
            _sample = pd.concat([_sample, _dead_sample], axis=1)
       
        n += NUM_EVAL_SAMPLES
        evaluation = equation.evaluate(_sample)
        bad_indices = np.where(np.isnan(evaluation) | np.isinf(evaluation))[0]
        _sample = _sample.drop(bad_indices)
        if np.isnan(evaluation).any() or np.isinf(evaluation).any():
            i+=len(bad_indices)
        conditions_ = pd.concat([conditions_,_sample], ignore_index=True)
    if i >= 1_000_000:
        return None
    conditions_ = conditions_.head(NUM_EVAL_SAMPLES)   
    experiment_data = experiment.experiment_runner(conditions_, added_noise = 0.0)
    X, y = experiment_data[ivs], experiment_data[dvs]
    prediction_bms = models_bms[-1].predict(X)
    prediction_linear = models_linear[-1].predict(X)
    prediction_polynomial = models_polynom[-1].predict(X)
    mad_bms_oos = mean_absolute_error(y, prediction_bms)
    mad_linear_oos = mean_absolute_error(y, prediction_linear)
    mad_poly_oos = mean_absolute_error(y, prediction_polynomial)
    msd_bms_oos = mean_squared_error(y, prediction_bms)
    msd_linear_oos = mean_squared_error(y, prediction_linear)
    msd_poly_oos = mean_squared_error(y, prediction_polynomial)
    mxe_bms_oos = max_error(y, prediction_bms)
    mxe_linear_oos = max_error(y, prediction_linear)
    mxe_poly_oos = max_error(y, prediction_polynomial)
    return Delta(mad_bms_oos = [mad_bms_oos], mad_linear_oos = [mad_linear_oos], mad_poly_oos = [mad_poly_oos], msd_bms_oos = [msd_bms_oos], msd_linear_oos = [msd_linear_oos], msd_poly_oos = [msd_poly_oos], mxe_bms_oos = [mxe_bms_oos], mxe_linear_oos = [mxe_linear_oos], mxe_poly_oos = [mxe_poly_oos])

### Cycle

In [68]:
def cycle(s):
    #if len(DEAD_VARIABLES)>0:
    #    s_pool = experimentalist_pooler(s, equation=equation)#, dead_variables=DEAD_VARIABLES)
    #else:
    s_pool = experimentalist_pooler(s, equation=equation)
    if ANNEALING: 
        
        temperature_now = cos_annealed_t(TEMPERATURE_0, CYCLES, s.cycle_step)
        # temperature is lower bounded by TEMPERATURE
        if temperature_now < TEMPERATURE:
            temperature_now = TEMPERATURE
        s_conditions = experimentalist_sample(s_pool, temperature=temperature_now, weights=WEIGHTS, num_samples=NUM_SAMPLES)
    else:
        s_conditions = experimentalist_sample(s_pool, temperature=TEMPERATURE, weights=WEIGHTS, num_samples=NUM_SAMPLES)
    
    s_run = runner_on_state(s_conditions, added_noise = ADDED_NOISE)
    #print(s_run)
    s_theory = bms_theorist(s_run)
    s_theory = linear_theorist(s_theory)
    s_theory = polynomial_theorist(s_theory)
    s_best = best_model(s_theory)
    s_oos = evaluation_oos(s_best)
    # WE NEED TO IMPLEMENT TESTING ON OUT OF SAMPLE DATA
    return s_oos


In [69]:
state = ExtendedState(variables=variables)
for _ in range(CYCLES):
    state = cycle(state)
    print(state)


INFO:autora.theorist.bms.regressor:BMS fitting started


First cycle: Using random sampler


100%|██████████| 100/100 [00:08<00:00, 11.26it/s]
INFO:autora.theorist.bms.regressor:BMS fitting finished


ExtendedState(variables=VariableCollection(independent_variables=[Variable(name='x_1', value_range=(-5, 5), allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False), Variable(name='d_1', value_range=(-5, 5), allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], dependent_variables=[Variable(name='y', value_range=None, allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], covariates=[]), conditions=           x_1       d_1
6231 -3.078344 -3.875316
3702 -1.600468 -0.666894
326   2.930298 -4.240452
8691 -2.481627  2.979559
1688 -1.669824 -3.798361
...        ...       ...
4006 -3.548799 -0.914915
1694  3.207683  3.933749
8904 -2.798100  2.417094
1585 -2.328911  0.224295
9326  4.297390  0.383257

[100 rows x 2 columns], experiment_data=           x_1       d_1         y
6231 -3.078344 -3.875316 -1.734497
3702 -1.60046

INFO:autora.theorist.bms.regressor:BMS fitting started
100%|██████████| 100/100 [00:08<00:00, 11.30it/s]
INFO:autora.theorist.bms.regressor:BMS fitting finished


...

INFO:autora.theorist.bms.regressor:BMS fitting started
100%|██████████| 100/100 [00:09<00:00, 10.62it/s]
INFO:autora.theorist.bms.regressor:BMS fitting finished


ExtendedState(variables=VariableCollection(independent_variables=[Variable(name='x_1', value_range=(-5, 5), allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False), Variable(name='d_1', value_range=(-5, 5), allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], dependent_variables=[Variable(name='y', value_range=None, allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], covariates=[]), conditions=           x_1       d_1
9442 -2.514051  1.892465
842  -2.177854 -0.892541
6730 -1.830385 -3.174264
6597 -1.892318 -4.048190
2933  2.862880 -3.595122
...        ...       ...
8316  4.857158  0.895806
7356  3.195319  1.874823
5806  2.652326  4.505137
8004  0.737025 -4.248729
1888  4.649484  4.180005

[100 rows x 2 columns], experiment_data=           x_1       d_1         y
0    -3.078344 -3.875316 -1.734497
1    -1.60046

INFO:autora.theorist.bms.regressor:BMS fitting started
100%|██████████| 100/100 [00:12<00:00,  7.74it/s]
INFO:autora.theorist.bms.regressor:BMS fitting finished


ExtendedState(variables=VariableCollection(independent_variables=[Variable(name='x_1', value_range=(-5, 5), allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False), Variable(name='d_1', value_range=(-5, 5), allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], dependent_variables=[Variable(name='y', value_range=None, allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], covariates=[]), conditions=           x_1       d_1
6121  0.199294  4.854395
1451  0.027819  2.504949
4371  3.878162  2.654446
728   4.424843 -1.200484
2652  0.625856  4.231155
...        ...       ...
5897 -0.981895  0.935465
3574 -1.066975  2.160864
1074  2.827105  2.591867
395  -2.651752  4.487403
9132  3.661967  2.126091

[100 rows x 2 columns], experiment_data=           x_1       d_1         y
0    -3.078344 -3.875316 -1.734497
1    -1.60046

KeyboardInterrupt: 